In [1]:
from server import *
from global_functions import *
from analysis.analysis import *
from games.guessing_game import *
from games.bar_game import *
from games.diner_dilemma import *
from games.divide_dollar import *
from games.sealed_bid_auction import *
from games.battle_royale import *
from games.public_goods import *
from games.pirate_game import *

In [25]:
# 0125 default

models = ["gpt-3.5-turbo_0613", "gpt-3.5-turbo_1106","gpt-3.5-turbo_0125", "gpt-4", "gpt-4o", "gemini-1.0", "gemini-1.5"]
name = {
    "gpt-3.5-turbo_0613": "gpt-3.5-turbo-0613_",
    "gpt-3.5-turbo_1106": "gpt-3.5-turbo-1106_",
    "gpt-3.5-turbo_0125": "", 
    "gpt-4": "gpt-4-0125-preview_",
    "gpt-4o": "gpt-4o_", 
    "gemini-1.0": "gemini-1.0-pro_",
    "gemini-1.5": "gemini-1.5-pro_",
}
gamename = ["guessing_game", "bar_game_implicit", "divide_dollar", "public_goods", "diner_dilemma", "sealed_bid_auction", "battle_royale", "pirate_game"]

gamedict = {
    "guessing_game": "Guess 2/3 of the Average", 
    "bar_game_implicit": "El Farol Bar", 
    "divide_dollar": "Divide the Dollar",
    "public_goods": "Public Goods Game",
    "diner_dilemma": "Diner’s Dilemma", 
    "sealed_bid_auction": "Sealed-Bid Auction",
    "battle_royale": "Battle Royale",
    "pirate_game": "Pirate Game"
}

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
row_score = []
game_row = []
game_score_dict = defaultdict(list)

for game in gamename:
    game_row_score = []
    for model in models:
    # Collect scores for each game
        scores = []  # Reset scores for each game
        # clear the score of the 5 runs of the game for each model
        game_scores = []
        for i in range(1,6):
            if game == "guessing_game":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', GuessingGame)
            elif game == "bar_game_implicit":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', BarGame)
            elif game == "diner_dilemma":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', DinerDilemma)
            elif game == "divide_dollar":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', DivideDollar)
            elif game == "sealed_bid_auction":
                gamex = load(f'raw_results/{game}/first_price/{name[model]}{game}_v1_{i}.json', SealedBidAuction)
            elif game == "public_goods":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', PublicGoods)
            elif game == "pirate_game":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', PirateGame)
            elif game == "battle_royale":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', BattleRoyale)
            x = gamex.compute_score()
            scores.append(x)
        # store all the scores of the 5 run of a game
        game_scores.append(scores)
        # store the 5 run scores of each game for each model
        game_score_dict[model].append(game_scores)
        # Calculate average and standard deviation with one significant figure
        average = round(np.mean(scores), 1)
        std_dev = round(np.std(scores, ddof=1), 1)
        game_row_score.append(f"${average}_{{\pm{std_dev}}}$")
    game_row.append([gamedict[game]] + [s for s in game_row_score])

game_averages = []
for k, game_score_list in game_score_dict.items():
    # print(k, game_score_list)
    game_averages.append([round(np.mean(np.mean(game_score_list, axis=0)), 1), round(np.std(np.mean(game_score_list, axis=0),ddof=1), 1)])

# import pprint
# pprint.pprint(row_score)
game_row.append(["Overall"] + [f"${round(avg, 1)}_{{\pm{round(sd, 1)}}}$" for avg, sd in game_averages])

df = pd.DataFrame(game_row)

# Hide index
df = df.style.hide(axis='index')
print(df.to_latex())
df

[[42.7, 2.0]]
[[42.7, 2.0], [45.1, 1.6]]
[[42.7, 2.0], [45.1, 1.6], [44.4, 2.1]]
[[42.7, 2.0], [45.1, 1.6], [44.4, 2.1], [62.4, 2.7]]
[[42.7, 2.0], [45.1, 1.6], [44.4, 2.1], [62.4, 2.7], [66.7, 4.7]]
[[42.7, 2.0], [45.1, 1.6], [44.4, 2.1], [62.4, 2.7], [66.7, 4.7], [45.7, 3.4]]
[[42.7, 2.0], [45.1, 1.6], [44.4, 2.1], [62.4, 2.7], [66.7, 4.7], [45.7, 3.4], [69.8, 1.6]]
[['Guess 2/3 of the Average', '$41.4_{\\pm0.5}$', '$68.5_{\\pm0.5}$', '$63.4_{\\pm3.4}$', '$91.6_{\\pm0.6}$', '$94.3_{\\pm0.6}$', '$77.3_{\\pm6.2}$', '$95.4_{\\pm0.5}$'], ['El Farol Bar', '$74.8_{\\pm4.5}$', '$64.3_{\\pm3.1}$', '$68.7_{\\pm2.7}$', '$23.0_{\\pm8.0}$', '$70.0_{\\pm22.1}$', '$33.5_{\\pm10.3}$', '$37.2_{\\pm4.2}$'], ['Divide the Dollar', '$42.4_{\\pm7.7}$', '$70.3_{\\pm3.3}$', '$68.6_{\\pm2.4}$', '$98.1_{\\pm1.9}$', '$95.2_{\\pm0.7}$', '$77.6_{\\pm3.6}$', '$93.8_{\\pm0.3}$'], ['Public Goods Game', '$17.7_{\\pm1.7}$', '$43.5_{\\pm12.6}$', '$38.9_{\\pm8.1}$', '$89.2_{\\pm1.8}$', '$90.9_{\\pm3.0}$', '$68.5_{\\pm

0,1,2,3,4,5,6,7
Guess 2/3 of the Average,$41.4_{\pm0.5}$,$68.5_{\pm0.5}$,$63.4_{\pm3.4}$,$91.6_{\pm0.6}$,$94.3_{\pm0.6}$,$77.3_{\pm6.2}$,$95.4_{\pm0.5}$
El Farol Bar,$74.8_{\pm4.5}$,$64.3_{\pm3.1}$,$68.7_{\pm2.7}$,$23.0_{\pm8.0}$,$70.0_{\pm22.1}$,$33.5_{\pm10.3}$,$37.2_{\pm4.2}$
Divide the Dollar,$42.4_{\pm7.7}$,$70.3_{\pm3.3}$,$68.6_{\pm2.4}$,$98.1_{\pm1.9}$,$95.2_{\pm0.7}$,$77.6_{\pm3.6}$,$93.8_{\pm0.3}$
Public Goods Game,$17.7_{\pm1.7}$,$43.5_{\pm12.6}$,$38.9_{\pm8.1}$,$89.2_{\pm1.8}$,$90.9_{\pm3.0}$,$68.5_{\pm7.6}$,$100.0_{\pm0.0}$
Diner’s Dilemma,$67.0_{\pm4.9}$,$1.4_{\pm1.3}$,$2.8_{\pm2.8}$,$0.9_{\pm0.7}$,$10.7_{\pm8.3}$,$3.1_{\pm1.5}$,$35.9_{\pm5.3}$
Sealed-Bid Auction,$10.3_{\pm0.2}$,$7.6_{\pm1.8}$,$13.0_{\pm1.5}$,$24.2_{\pm1.1}$,$20.8_{\pm3.2}$,$31.6_{\pm12.2}$,$26.9_{\pm9.4}$
Battle Royale,$19.5_{\pm7.7}$,$35.7_{\pm6.8}$,$28.6_{\pm11.0}$,$86.8_{\pm9.7}$,$67.3_{\pm14.8}$,$16.5_{\pm6.9}$,$81.3_{\pm7.7}$
Pirate Game,$68.4_{\pm19.9}$,$69.5_{\pm14.6}$,$71.6_{\pm7.7}$,$85.4_{\pm8.7}$,$84.4_{\pm6.7}$,$57.4_{\pm14.3}$,$87.9_{\pm5.6}$
Overall,$42.7_{\pm2.0}$,$45.1_{\pm1.6}$,$44.4_{\pm2.1}$,$62.4_{\pm2.7}$,$66.7_{\pm4.7}$,$45.7_{\pm3.4}$,$69.8_{\pm1.6}$


In [61]:
models = ["llama-3.1-8b", "llama-3.1-70b", "llama-3.1-405b", "mixtral8x7b", "mixtral8x22b", "qwen2-72b"]
name = {
    "llama-3.1-8b": "llama-3.1-8b_",
    "llama-3.1-70b": "llama-3.1-70b_",
    "llama-3.1-405b": "llama-3.1-405b_",
    "mixtral8x7b": "mixtral-8x7b_",
    "mixtral8x22b": "mixtral-8x22b_",
    "qwen2-72b": "qwen2-72b_"
}
gamename = ["guessing_game", "bar_game_implicit", "divide_dollar", "public_goods", "diner_dilemma", "sealed_bid_auction", "battle_royale", "pirate_game"]
gamedict = {
    "guessing_game": "Guess 2/3 of the Average", 
    "bar_game_implicit": "El Farol Bar", 
    "divide_dollar": "Divide the Dollar",
    "public_goods": "Public Goods Game",
    "diner_dilemma": "Diner’s Dilemma", 
    "sealed_bid_auction": "Sealed-Bid Auction",
    "battle_royale": "Battle Royale",
    "pirate_game": "Pirate Game"
}

In [63]:
import pandas as pd
import numpy as np
from collections import defaultdict
row_score = []
game_row = []
game_score_dict = defaultdict(list)

for game in gamename:
    game_row_score = []
    for model in models:
    # Collect scores for each game
        scores = []  # Reset scores for each game
        # clear the score of the 5 runs of the game for each model
        game_scores = []
        for i in range(1,6):
            if game == "guessing_game":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', GuessingGame)
            elif game == "bar_game_implicit":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', BarGame)
            elif game == "diner_dilemma":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', DinerDilemma)
            elif game == "divide_dollar":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', DivideDollar)
            elif game == "sealed_bid_auction":
                gamex = load(f'raw_results/{game}/first_price/{name[model]}{game}_v1_{i}.json', SealedBidAuction)
            elif game == "public_goods":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', PublicGoods)
            elif game == "pirate_game":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', PirateGame)
            elif game == "battle_royale":
                gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', BattleRoyale)
            x = gamex.compute_score()
            scores.append(x)
        # store all the scores of the 5 run of a game
        game_scores.append(scores)
        # store the 5 run scores of each game for each model
        game_score_dict[model].append(game_scores)
        # Calculate average and standard deviation with one significant figure
        average = round(np.mean(scores), 1)
        std_dev = round(np.std(scores, ddof=1), 1)
        game_row_score.append(f"${average}_{{\pm{std_dev}}}$")
    game_row.append([gamedict[game]] + [s for s in game_row_score])

game_averages = []
for k, game_score_list in game_score_dict.items():
    # print(k, game_score_list)
    game_averages.append([round(np.mean(np.mean(game_score_list, axis=0)), 1), round(np.std(np.mean(game_score_list, axis=0),ddof=1), 1)])

# import pprint
# pprint.pprint(row_score)
game_row.append(["Overall"] + [f"${round(avg, 1)}_{{\pm{round(sd, 1)}}}$" for avg, sd in game_averages])

df = pd.DataFrame(game_row)

# Hide index
df = df.style.hide(axis='index')
print(df.to_latex())
df

\begin{tabular}{lllllll}
0 & 1 & 2 & 3 & 4 & 5 & 6 \\
Guess 2/3 of the Average & $85.5_{\pm3.0}$ & $84.0_{\pm1.7}$ & $94.3_{\pm0.6}$ & $91.8_{\pm0.4}$ & $83.6_{\pm4.6}$ & $93.2_{\pm1.3}$ \\
El Farol Bar & $75.7_{\pm2.2}$ & $59.7_{\pm3.5}$ & $20.5_{\pm24.2}$ & $66.8_{\pm5.8}$ & $39.3_{\pm12.2}$ & $17.0_{\pm25.5}$ \\
Divide the Dollar & $56.4_{\pm8.4}$ & $87.0_{\pm4.1}$ & $94.9_{\pm1.0}$ & $1.2_{\pm2.8}$ & $79.0_{\pm9.6}$ & $91.9_{\pm2.4}$ \\
Public Goods Game & $19.6_{\pm1.0}$ & $90.6_{\pm3.6}$ & $97.0_{\pm0.8}$ & $27.6_{\pm11.7}$ & $83.7_{\pm3.5}$ & $81.3_{\pm5.9}$ \\
Diner’s Dilemma & $59.3_{\pm2.4}$ & $48.1_{\pm5.7}$ & $14.4_{\pm4.5}$ & $76.4_{\pm7.1}$ & $79.9_{\pm5.8}$ & $0.0_{\pm0.0}$ \\
Sealed-Bid Auction & $37.1_{\pm3.1}$ & $15.7_{\pm2.7}$ & $14.7_{\pm3.2}$ & $3.1_{\pm1.6}$ & $13.2_{\pm3.7}$ & $2.5_{\pm0.7}$ \\
Battle Royale & $35.9_{\pm12.1}$ & $77.7_{\pm26.0}$ & $92.7_{\pm10.1}$ & $12.6_{\pm9.4}$ & $36.0_{\pm21.0}$ & $81.7_{\pm9.6}$ \\
Pirate Game & $78.3_{\pm10.0}$ & $64.0_{\p

0,1,2,3,4,5,6
Guess 2/3 of the Average,$85.5_{\pm3.0}$,$84.0_{\pm1.7}$,$94.3_{\pm0.6}$,$91.8_{\pm0.4}$,$83.6_{\pm4.6}$,$93.2_{\pm1.3}$
El Farol Bar,$75.7_{\pm2.2}$,$59.7_{\pm3.5}$,$20.5_{\pm24.2}$,$66.8_{\pm5.8}$,$39.3_{\pm12.2}$,$17.0_{\pm25.5}$
Divide the Dollar,$56.4_{\pm8.4}$,$87.0_{\pm4.1}$,$94.9_{\pm1.0}$,$1.2_{\pm2.8}$,$79.0_{\pm9.6}$,$91.9_{\pm2.4}$
Public Goods Game,$19.6_{\pm1.0}$,$90.6_{\pm3.6}$,$97.0_{\pm0.8}$,$27.6_{\pm11.7}$,$83.7_{\pm3.5}$,$81.3_{\pm5.9}$
Diner’s Dilemma,$59.3_{\pm2.4}$,$48.1_{\pm5.7}$,$14.4_{\pm4.5}$,$76.4_{\pm7.1}$,$79.9_{\pm5.8}$,$0.0_{\pm0.0}$
Sealed-Bid Auction,$37.1_{\pm3.1}$,$15.7_{\pm2.7}$,$14.7_{\pm3.2}$,$3.1_{\pm1.6}$,$13.2_{\pm3.7}$,$2.5_{\pm0.7}$
Battle Royale,$35.9_{\pm12.1}$,$77.7_{\pm26.0}$,$92.7_{\pm10.1}$,$12.6_{\pm9.4}$,$36.0_{\pm21.0}$,$81.7_{\pm9.6}$
Pirate Game,$78.3_{\pm10.0}$,$64.0_{\pm15.5}$,$65.6_{\pm22.3}$,$67.3_{\pm7.6}$,$84.3_{\pm8.8}$,$86.1_{\pm6.4}$
Overall,$56.0_{\pm3.1}$,$65.9_{\pm3.3}$,$61.8_{\pm4.7}$,$43.4_{\pm2.2}$,$62.4_{\pm2.2}$,$56.7_{\pm3.4}$


In [ ]:
models = ["gpt-3.5-turbo_", "gpt-4o_", "gemini-1.5-pro_", "llama-3.1-70b_"]
model = models[0]
name = {
    "gpt-3.5-turbo_": "", 
    "gpt-4o_": "gpt-4o_", 
    "gemini-1.5-pro_": "gemini-1.5-pro_", 
    "llama-3.1-70b_": "llama-3.1-70b_"
}
"bar_game_implicit", 
gamename = ["guessing_game", "bar_game_implicit", "divide_dollar", "public_goods", "diner_dilemma", "sealed_bid_auction", "battle_royale", "pirate_game"]
gamedict = {
    "guessing_game": "Guess 2/3 of the Average", 
    "bar_game_implicit": "El Farol Bar", 
    "divide_dollar": "Divide the Dollar",
    "public_goods": "Public Goods Game",
    "diner_dilemma": "Diner’s Dilemma", 
    "sealed_bid_auction": "Sealed-Bid Auction",
    "battle_royale": "Battle Royale",
    "pirate_game": "Pirate Game"
}

In [64]:
import pandas as pd
import numpy as np
data = []
game_scores = []
# Collect scores for each game
for game in gamename:
    scores = []  # Reset scores for each game
    for i in range(1,6):
        if game == "guessing_game":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', GuessingGame)
        elif game == "bar_game_implicit":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', BarGame)
        elif game == "diner_dilemma":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', DinerDilemma)
        elif game == "divide_dollar":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', DivideDollar)
        elif game == "sealed_bid_auction":
            gamex = load(f'raw_results/{game}/first_price/{name[model]}{game}_v1_{i}.json', SealedBidAuction)
        elif game == "public_goods":
            print("goods")
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', PublicGoods)
        elif game == "pirate_game":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', PirateGame)
        elif game == "battle_royale":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_{i}.json', BattleRoyale)
        x = gamex.compute_score()
        scores.append(x)
    game_scores.append(scores)
    # Calculate average and standard deviation with one significant figure
    average = round(np.mean(scores), 1)
    std_dev = round(np.std(scores, ddof=1), 1)
    scores = [f"${round(s, 1)}$" for s in scores]
    # Append row data for the current game
    data.append([gamedict[game]] + scores + [f"${average}_{{\pm{std_dev}}}$"])
game_column_mean_score = np.mean(game_scores, axis=0)
game_overall_mean_score = round(np.mean(game_column_mean_score), 1)
game_overall_std = round(np.std(game_column_mean_score,ddof=1,axis=0) ,1)
data.append(["\\bf Overall"] + [f"${round(s, 1)}$" for s in game_column_mean_score] + [f"${game_overall_mean_score}_{{\pm{game_overall_std}}}$"])
print(game_column_mean_score)
print(game_overall_mean_score)
print(game_overall_std)
# data.append("Overall" + game_overall_score + [f"${average}_{{\pm{std_dev}}}$"]))
# Create DataFrame
columns = [r"\bf Tests", r"\bf T1 (Default)", r"\bf T2", r"\bf T3", r"\bf T4", r"\bf T5", r"\bf $Avg_{\pm Std}$"]
df = pd.DataFrame(data, columns=columns)

# Hide index
df = df.style.hide(axis='index')
print(df.to_latex())
df

goods
goods
goods
goods
goods
[56.03673195 56.08789252 54.6797782  54.15837115 62.57266541]
56.7
3.4
\begin{tabular}{lllllll}
\bf Tests & \bf T1 (Default) & \bf T2 & \bf T3 & \bf T4 & \bf T5 & \bf $Avg_{\pm Std}$ \\
Guess 2/3 of the Average & $91.2$ & $94.2$ & $92.7$ & $94.1$ & $93.7$ & $93.2_{\pm1.3}$ \\
El Farol Bar & $18.3$ & $1.7$ & $2.5$ & $1.7$ & $60.8$ & $17.0_{\pm25.5}$ \\
Divide the Dollar & $92.2$ & $87.8$ & $92.0$ & $93.8$ & $93.7$ & $91.9_{\pm2.4}$ \\
Public Goods Game & $82.5$ & $85.8$ & $79.7$ & $86.5$ & $72.0$ & $81.3_{\pm5.9}$ \\
Diner’s Dilemma & $0.0$ & $0.0$ & $0.0$ & $0.0$ & $0.0$ & $0.0_{\pm0.0}$ \\
Sealed-Bid Auction & $3.3$ & $3.1$ & $2.2$ & $1.6$ & $2.4$ & $2.5_{\pm0.7}$ \\
Battle Royale & $85.0$ & $90.0$ & $75.0$ & $68.4$ & $90.0$ & $81.7_{\pm9.6}$ \\
Pirate Game & $75.8$ & $86.2$ & $93.4$ & $87.2$ & $87.9$ & $86.1_{\pm6.4}$ \\
\bf Overall & $56.0$ & $56.1$ & $54.7$ & $54.2$ & $62.6$ & $56.7_{\pm3.4}$ \\
\end{tabular}



\bf Tests,\bf T1 (Default),\bf T2,\bf T3,\bf T4,\bf T5,\bf $Avg_{\pm Std}$
Guess 2/3 of the Average,$91.2$,$94.2$,$92.7$,$94.1$,$93.7$,$93.2_{\pm1.3}$
El Farol Bar,$18.3$,$1.7$,$2.5$,$1.7$,$60.8$,$17.0_{\pm25.5}$
Divide the Dollar,$92.2$,$87.8$,$92.0$,$93.8$,$93.7$,$91.9_{\pm2.4}$
Public Goods Game,$82.5$,$85.8$,$79.7$,$86.5$,$72.0$,$81.3_{\pm5.9}$
Diner’s Dilemma,$0.0$,$0.0$,$0.0$,$0.0$,$0.0$,$0.0_{\pm0.0}$
Sealed-Bid Auction,$3.3$,$3.1$,$2.2$,$1.6$,$2.4$,$2.5_{\pm0.7}$
Battle Royale,$85.0$,$90.0$,$75.0$,$68.4$,$90.0$,$81.7_{\pm9.6}$
Pirate Game,$75.8$,$86.2$,$93.4$,$87.2$,$87.9$,$86.1_{\pm6.4}$
\bf Overall,$56.0$,$56.1$,$54.7$,$54.2$,$62.6$,$56.7_{\pm3.4}$


In [ ]:
import pandas as pd
import numpy as np
data = []

# Collect scores for each game
for game in gamename:
    scores = []  # Reset scores for each game
    for i in range(6):
        if game == "guessing_game":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_t{i}.json', GuessingGame)
        elif game == "bar_game_implicit":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_t{i}.json', BarGame)
        elif game == "diner_dilemma":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_t{i}.json', DinerDilemma)
        elif game == "divide_dollar":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_t{i}.json', DivideDollar)
        x = gamex.compute_score()
        scores.append(x)

    # Calculate average and standard deviation with one significant figure
    average = round(np.mean(scores), 1)
    std_dev = round(np.std(scores, ddof=1), 1)
    scores = [f"${round(s, 1)}$" for s in scores]

    # Append row data for the current game
    data.append([gamedict[game]] + scores + [f"${average}_{{\pm{std_dev}}}$"])

# Create DataFrame
columns = [r"\bf Temperature", r"\bf 0.0", r"\bf 0.2", r"\bf 0.4", r"\bf 0.6", r"\bf 0.8", r"\bf 1.0", r"\bf $Avg_{\pm Std}$"]
df = pd.DataFrame(data, columns=columns)

# Hide index
df = df.style.hide(axis='index')
print(df.to_latex())
df

[['Guess 2/3 of the Average', '$96.1$', '$99.2$', '$96.6$', '$96.6$', '$96.4$', '$96.2$', '$96.9_{\\pm1.2}$'], ['El Farol Bar', '$37.5$', '$20.0$', '$28.3$', '$40.0$', '$38.3$', '$37.5$', '$33.6_{\\pm7.8}$'], ['Divide the Dollar', '$94.5$', '$93.5$', '$93.5$', '$94.5$', '$93.3$', '$93.8$', '$93.8_{\\pm0.5}$'], ['Public Goods Game', '$93.8$', '$93.8$', '$93.8$', '$93.8$', '$93.8$', '$93.8$', '$93.8_{\\pm0.0}$'], ['Diner’s Dilemma', '$33.5$', '$45.0$', '$43.0$', '$36.5$', '$42.0$', '$29.0$', '$38.2_{\\pm6.2}$'], ['Sealed-Bid Auction', '$29.0$', '$29.0$', '$29.0$', '$29.0$', '$29.0$', '$29.0$', '$29.0_{\\pm0.0}$'], ['Battle Royale', '$29.0$', '$29.0$', '$29.0$', '$29.0$', '$29.0$', '$29.0$', '$29.0_{\\pm0.0}$'], ['Pirate Game', '$29.0$', '$29.0$', '$29.0$', '$29.0$', '$29.0$', '$29.0$', '$29.0_{\\pm0.0}$']]
\begin{tabular}{llllllll}
\bf Temperature & \bf 0.0 & \bf 0.2 & \bf 0.4 & \bf 0.6 & \bf 0.8 & \bf 1.0 & \bf $Avg_{\pm Std}$ \\
Guess 2/3 of the Average & $96.1$ & $99.2$ & $96.6$ & $96

\bf Temperature,\bf 0.0,\bf 0.2,\bf 0.4,\bf 0.6,\bf 0.8,\bf 1.0,\bf $Avg_{\pm Std}$
Guess 2/3 of the Average,$96.1$,$99.2$,$96.6$,$96.6$,$96.4$,$96.2$,$96.9_{\pm1.2}$
El Farol Bar,$37.5$,$20.0$,$28.3$,$40.0$,$38.3$,$37.5$,$33.6_{\pm7.8}$
Divide the Dollar,$94.5$,$93.5$,$93.5$,$94.5$,$93.3$,$93.8$,$93.8_{\pm0.5}$
Public Goods Game,$93.8$,$93.8$,$93.8$,$93.8$,$93.8$,$93.8$,$93.8_{\pm0.0}$
Diner’s Dilemma,$33.5$,$45.0$,$43.0$,$36.5$,$42.0$,$29.0$,$38.2_{\pm6.2}$
Sealed-Bid Auction,$29.0$,$29.0$,$29.0$,$29.0$,$29.0$,$29.0$,$29.0_{\pm0.0}$
Battle Royale,$29.0$,$29.0$,$29.0$,$29.0$,$29.0$,$29.0$,$29.0_{\pm0.0}$
Pirate Game,$29.0$,$29.0$,$29.0$,$29.0$,$29.0$,$29.0$,$29.0_{\pm0.0}$


In [ ]:
import pandas as pd
import numpy as np
data = []

# Collect scores for each game
for game in gamename:
    scores = []  # Reset scores for each game
    for i in range(1, 6):
        if game == "guessing_game":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v{i}.json', GuessingGame)
        elif game == "bar_game_implicit":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v{i}.json', BarGame)
        elif game == "diner_dilemma":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v{i}.json', DinerDilemma)
        elif game == "divide_dollar":
            gamex = load(f'raw_results/{game}/{name[model]}{game}_v{i}.json', DivideDollar)
        x = gamex.compute_score()
        scores.append(x)

    # Calculate average and standard deviation with one significant figure
    average = round(np.mean(scores), 1)
    std_dev = round(np.std(scores, ddof=1), 1)
    scores = [f"${round(s, 1)}$" for s in scores]

    # Append row data for the current game
    data.append([gamedict[game]] + scores + [f"${average}_{{\pm{std_dev}}}$"])

# Create DataFrame
columns = [r"\bf Version", r"\bf V1 (Default)", r"\bf V2", r"\bf V3", r"\bf V4", r"\bf V5", r"\bf $Avg_{\pm Std}$"]
df = pd.DataFrame(data, columns=columns)

# Hide index
df = df.style.hide(axis='index')
print(df.to_latex())
df

In [ ]:
import pandas as pd
import numpy as np
data = []

print(model)
game = gamename[0]
scores = []
for i in range(13):
    gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_r{i}.json', GuessingGame)
    x = gamex.compute_score()
    scores.append(x)
average = round(np.mean(scores), 1)
std_dev = round(np.std(scores, ddof=1), 1)
scores = [f"${round(s, 1)}$" for s in scores]
data.append([gamedict[game]] + scores + [f"${average}_{{\pm{std_dev}}}$"])

# columns = [r"\bf Game"] + [f"Round {i+1}" for i in range(13)] + [r"$\bf Avg_{\pm Std}$"]

# Create DataFrame
df = pd.DataFrame(data)

# Hide index
df = df.style.hide(axis='index')
print(df.to_latex())
df

In [ ]:
import pandas as pd
import numpy as np
data = []

print(model)
game = gamename[1]
scores = []
for i in range(6):
    gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_r{i}.json', BarGame)
    x = gamex.compute_score()
    scores.append(x)
average = round(np.mean(scores), 1)
std_dev = round(np.std(scores, ddof=1), 1)
scores = [f"${round(s, 1)}$" for s in scores]
data.append([gamedict[game]] + scores + [f"${average}_{{\pm{std_dev}}}$"])

# columns = [r"\bf Game"] + [f"Round {i+1}" for i in range(13)] + [r"$\bf Avg_{\pm Std}$"]

# Create DataFrame
df = pd.DataFrame(data)

# Hide index
df = df.style.hide(axis='index')
print(df.to_latex())
df

In [ ]:
import pandas as pd
import numpy as np
data = []

print(model)
game = gamename[2]
scores = []
for i in range(1,6):
    gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_g{i}.json', DivideDollar)
    x = gamex.compute_score()
    scores.append(x)
average = round(np.mean(scores), 1)
std_dev = round(np.std(scores, ddof=1), 1)
scores = [f"${round(s, 1)}$" for s in scores]
data.append([gamedict[game]] + scores + [f"${average}_{{\pm{std_dev}}}$"])

# columns = [r"\bf Game"] + [f"Round {i+1}" for i in range(13)] + [r"$\bf Avg_{\pm Std}$"]

# Create DataFrame
df = pd.DataFrame(data)

# Hide index
df = df.style.hide(axis='index')
print(df.to_latex())
df

In [ ]:
import pandas as pd
import numpy as np
data = []

print(model)
game = gamename[3]
scores = []
for i in range(1,7):
    gamex = load(f'raw_results/{game}/{name[model]}{game}_v1_p{i}.json', DinerDilemma)
    x = gamex.compute_score()
    scores.append(x)
average = round(np.mean(scores), 1)
std_dev = round(np.std(scores, ddof=1), 1)
scores = [f"${round(s, 1)}$" for s in scores]
data.append([gamedict[game]] + scores + [f"${average}_{{\pm{std_dev}}}$"])

# columns = [r"\bf Game"] + [f"Round {i+1}" for i in range(13)] + [r"$\bf Avg_{\pm Std}$"]

# Create DataFrame
df = pd.DataFrame(data)

# Hide index
df = df.style.hide(axis='index')
print(df.to_latex())
df